In [32]:
source('../utilities.R')
library(readr)
quantile_forecast <- readRDS("../../Outputs/quantile_forecast.rds")
df <- read.csv("../../Data/palo_alto.csv")

In [33]:
starts = '2019-01-07'
ends = '2019-11-24' 
horizon = 14 
step = 15
tz = "US/Pacific"

In [34]:
sess <- sessions(df = df,train_starts = starts,train_ends = ends,tz = tz)
trans <- transactions(df = df,train_starts = starts,train_ends = ends,tz = tz)
prof <- load_profile(trans, step =  step,tz = "US/Pacific")

In [35]:
write_csv(prof %>% select(Start,Power),paste0('profiles_palo_alto.csv'))

In [36]:
#####

In [61]:
profile <- read_csv("profiles_palo_alto.csv") %>%
    mutate(Start = with_tz(Start,tzone=tz))

Rows: 30912 Columns: 2
── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
dbl  (1): Power
dttm (1): Start

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [76]:
calib_starts = as.Date('2019-09-30',tz=tz)
calib_ends = as.Date('2019-10-06',tz=tz) 
test_starts = as.Date('2019-10-07',tz=tz)
test_ends = as.Date('2019-10-13',tz=tz)
horizon = 14 
step = 15
calib_dates <- seq(from = calib_starts, to = calib_ends, by = "day")
test_dates <- seq(from = test_starts, to = test_ends, by = "day")
 
pred_dates <- seq(from = calib_starts, to = test_ends, by = "day")
 
observation <- profile  %>% filter(as.Date(Start,tz=tz) %in% pred_dates)
 
print("===== Conformal prediction =====")
# prediction  <- quantile_forecast %>% select(Start,mean)
prediction <- quantile_forecast %>% select(Start,q10,q90)
observation_calib <- observation %>% filter(as.Date(Start,tz=tz) %in% calib_dates)
observation_test  <- observation %>% filter(as.Date(Start,tz=tz) %in% test_dates)
prediction_calib  <- prediction %>% filter(as.Date(Start,tz=tz) %in% calib_dates)
prediction_test   <- prediction %>% filter(as.Date(Start,tz=tz) %in% test_dates)

[1] "===== Conformal prediction ====="


In [80]:
tail(observation_calib)
nrow(observation_calib)

Start,Power
<dttm>,<dbl>
2019-10-06 22:30:00,9.411546
2019-10-06 22:45:00,9.411546
2019-10-06 23:00:00,9.411546
2019-10-06 23:15:00,9.411546
2019-10-06 23:30:00,9.411546
2019-10-06 23:45:00,9.411546


[1] 672

In [79]:
tail(prediction_calib)
nrow(prediction_calib)

Start,q10,q90
<dttm>,<dbl>,<dbl>
2019-10-06 22:30:00,5.994910,35.65823
2019-10-06 22:45:00,5.860691,33.05179
2019-10-06 23:00:00,4.221708,30.28634
2019-10-06 23:15:00,3.332887,28.10652
2019-10-06 23:30:00,3.068357,26.51760
2019-10-06 23:45:00,0.000000,24.11753


[1] 672

### END